# import and Functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import signal
from scipy.fftpack import fft
from scipy import ndimage
from scipy.io import loadmat
from matplotlib.ticker import MaxNLocator
import os
import warnings
warnings.filterwarnings('ignore')
#annots = loadmat('C:\\Users\\llinc\\GitHub\\retina_personal\\0406\\merge\\merge_0224_HMM_RL_G2.5_5min_Q100_6.5mW.mat')
%matplotlib inline

In [2]:
## EqualState assign states with equal possibility for input array x
def EqualState(x, num_state):
    xs=np.sort(x)
    binlen=int(len(x)/num_state-0.5) #round
    edges = xs[np.arange(num_state+1)*binlen]
    xstate=np.zeros(len(x))
    for i in range(num_state):
        xstate[x>=edges[i]] = i
    xstate = xstate.astype(int)
    return xstate, edges

In [3]:
def PIfunc(r, x, v, dt, window):
    negshift=window[0] # second
    posshift=window[1] # second
    shiftlen=(posshift-negshift)/dt+1
    timeshift=np.linspace(negshift,posshift,int(shiftlen))
    bitshift=np.linspace(negshift/dt,posshift/dt,int(shiftlen),dtype = 'int16')
    Information = dict()
    Information[('BROJA_2PID','SI')]=np.zeros(len(bitshift))
    Information[('BROJA_2PID','UIx')]=np.zeros(len(bitshift))
    Information[('BROJA_2PID','UIv')]=np.zeros(len(bitshift))
    Information[('BROJA_2PID','CI')]=np.zeros(len(bitshift))
    Information[('Beer','Red')]=np.zeros(len(bitshift))
    Information[('Beer','UIx')]=np.zeros(len(bitshift))
    Information[('Beer','UIv')]=np.zeros(len(bitshift))
    Information[('Beer','Syn')]=np.zeros(len(bitshift))
    
    Information[('test','SI')]=np.zeros(len(bitshift))
    Information[('test','UIx')]=np.zeros(len(bitshift))
    Information[('test','UIv')]=np.zeros(len(bitshift))
    Information[('test','CI')]=np.zeros(len(bitshift))
    # shifted data
    # shift>0 => y shifted to positive side
    for i in range(len(bitshift)):
        xx=[]
        vv=[]
        rr=[]
        shift=bitshift[i]
        if shift>0:
            xx=x[shift:]
            vv=v[shift:]
            rr=r[:(-1*shift)]
        elif shift==0:
            xx=x
            vv=v
            rr=r
        elif shift<0:
            xx=x[:shift]
            vv=v[:shift]
            rr=r[(-1*shift):]
        #find weight of each states by 3D histogram 
        xedges = np.append(np.unique(xx),(max(xx)+1))
        vedges = np.append(np.unique(vv),(max(vv)+1))
        redges = np.append(np.unique(rr),(max(rr)+1))
        dat = np.concatenate((xx[:,np.newaxis], vv[:,np.newaxis],rr[:,np.newaxis]), axis=1)
        N, edges = np.histogramdd(dat, bins=(xedges, vedges, redges))
        #Calculate all kinds of probability and make sure the shape of them, 0 -> x, 1 -> v, 2 -> r
        px=(np.sum(N,axis=(1,2))/np.sum(N))[:, np.newaxis, np.newaxis]
        pv=(np.sum(N,axis=(0,2))/np.sum(N))[np.newaxis, :, np.newaxis]
        pr=(np.sum(N,axis=(0,1))/np.sum(N))[np.newaxis ,np.newaxis, :]
        pxv=(np.sum(N,axis=2)/np.sum(N))[:, :, np.newaxis]
        pxr=(np.sum(N,axis=1)/np.sum(N))[:, np.newaxis, :]
        pvr=(np.sum(N,axis=0)/np.sum(N))[np.newaxis, :, :]
        pxvr=(N/np.sum(N))
        
        Information[('test','UIx')][i] = np.nansum(pxvr*np.log2(pxvr*px/pxv/pxr))/dt
        Information[('test','UIv')][i] = np.nansum(pxvr*np.log2(pxvr*pv/pxv/pvr))/dt


        MIxr=np.nansum(pxr*np.log2(pxr/px/pr))/dt
        MIvr=np.nansum(pvr*np.log2(pvr/pv/pr))/dt
        MIxvR=np.nansum(pxvr*np.log2(pxvr/pxv/pr))/dt
        PI_xR = np.nansum(pxr*np.log2(pxr/px/pr), axis = (0,1))
        PI_vR = np.nansum(pvr*np.log2(pvr/pv/pr), axis = (0,1))
        R = sum(np.minimum(PI_xR, PI_vR))/dt
        Information[('Beer','Red')][i] = R
        Information[('Beer','UIx')][i] = MIxr - R
        Information[('Beer','UIv')][i] = MIvr - R
        Information[('Beer','Syn')][i] = MIxvR - MIxr - MIvr + R

    return timeshift, Information

In [4]:
def Spike_Time_Generater(rdt, dt, Garmma=1):
    rdt = rdt*Garmma
    Spike_time = []
    
    counter = 0
    post_remainer_c = 0
    p = 1
    while True:
        the_random_number = 1-np.random.rand()
        while (the_random_number < p and counter < len(rdt)):
            p *= np.exp(-rdt[counter])
            counter += 1
        if counter >= len(rdt):
            break
        remainer_c = -np.log(p/the_random_number)/rdt[counter-1]
#         if remainer_c>=1 or remainer_c<=0:
#             print('shit!')
        Spike_time.append(dt*(counter-remainer_c))
        p = np.exp(-remainer_c*rdt[counter-1])
    return Spike_time[::Garmma]

# activation function
def NL(x,theta=0):
    y = np.zeros(len(x))
    for i in range(len(x)):
        if x[i]- theta>0:
            y[i]= x[i]-theta
    return(y)

# chose file

In [5]:
dt = 0.01
Tot=300
T=np.arange(dt,Tot,dt)
tau= 1
D = 27*10**5
OU=np.zeros(len(T))
mu, sigma = 0, 1
for i in range(len(T)-1):
    OU[i+1]=OU[i]*(1-dt/tau)+math.sqrt(D*dt)*np.random.normal(mu,sigma)
OU = (OU-np.mean(OU))/np.std(OU)

# filtering for OUSmoothed
Fc = 1
b, a = signal.butter(2, 2*Fc*dt, btype='low', analog=False)
x = signal.filtfilt(b, a, OU)
x = x.astype(float)

In [6]:
x = x.astype(float)
x = (x -np.mean(x))/np.std(x)
x_cor = np.correlate(x, x, "same")
x_cor = x_cor[:int((len(x_cor)+1)/2)+1]
Taxis = np.flip(T[:len(x_cor)]-dt*3/2)
t_cor = np.interp(0.5*max(x_cor),  x_cor, Taxis)
tv = ndimage.gaussian_filter1d(x, sigma=int(t_cor/dt*0.1), order=1, mode='reflect') / dt
v=np.append(0, np.diff(x)/dt)
print(t_cor)

0.9028031001070052


# Set $\lambda$s

In [7]:
Lambdas =  np.append(np.arange(0.1,1.0,0.1),1.0)
betas = (1-Lambdas)/Lambdas
alpha = 40. #1/sec
K = 50.
phi = 10

In [8]:
# Lambdas = betas

In [9]:
##response
# Model one: simple estimation
window = [-1,1] # second
Information_list = []
dvInformation_list = []
y_list = []
z_list = []
for beta in betas:
    g = beta
    y=np.zeros(len(T))
    z=np.zeros(len(T))
    for j in range(len(T)-1):
        dy=dt*(-alpha*y[j]+K*(x[j]-phi*z[j]))
        dz=dt*(-beta*z[j]+g*y[j])
        y[j+1]=y[j]+dy
        z[j+1]=z[j]+dz
    y_list.append(y.copy())
    z_list.append(z.copy())

In [10]:
for i in range(len(y_list)):
    y = y_list[i]
    z = z_list[i]
    y_n = (y-np.mean(y))/np.std(y)
    z_n = (z-np.mean(z))/np.std(z)
    
    xy_cor = np.correlate(x, y_n, "same")
    yz_cor = np.correlate(y_n, z_n, "same")
    xz_cor = np.correlate(x, z_n, "same")
    xy_cor_time = round((T-np.mean(T))[np.argmax(xy_cor)], 3)
    yz_cor_time = round((T-np.mean(T))[np.argmax(yz_cor)], 3)
    xz_cor_time = round((T-np.mean(T))[np.argmax(xz_cor)], 3)
    print(xy_cor_time, yz_cor_time, xz_cor_time)

0.1 -0.11 -0.01
0.18 -0.2 -0.02
0.24 -0.27 -0.03
0.27 -0.33 -0.05
0.29 -0.37 -0.08
0.3 -0.42 -0.11
0.28 -0.47 -0.16
0.25 -0.57 -0.25
0.19 -0.83 -0.45
-0.02 -149.99 -149.99


In [11]:
for i in range(len(y_list)):
    y = y_list[i]
    z = z_list[i]
    print(np.std(y), np.std(z))

0.09406626608962623 0.09264385325278987
0.09992308930500407 0.09268102076461952
0.11297742367636705 0.09269002948115035
0.13667259022183575 0.0926328938985887
0.17538420127111856 0.09242751996742336
0.23526462588310024 0.0918891762117011
0.32628368489021975 0.09058116963075016
0.46772312509571834 0.08733668884352093
0.7120128807288942 0.07776265500252423
1.2493638229114716 0.0


In [12]:
y_n

array([4.14095939e-05, 2.02738236e-02, 4.12173557e-02, ...,
       4.98166409e-01, 4.97764094e-01, 4.97348242e-01])